# Test AD Models 

Test other Anomaly Detection models to compare to EIF+. 

Potential models to test: 

- Deep Isolation Forest (DIF) 
- INNE: Isolation Based AD using nearest neighbours ensemble
- SUOD: This is another ensemble model like EIF 
- Auto Encoder 

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from pyod.models.dif import DIF
from pyod.models.inne import INNE
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.suod import SUOD

from scipy.io import loadmat

sys.path.append('../src')
from src.performance_report_functions import *
from src.utils import *

sys.path.append('../models')
from models.forests import *

2024-02-09 14:37:28.787600: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load Data

In [15]:
os.chdir('../data/real/')
os.getcwd()

FileNotFoundError: [Errno 2] No such file or directory: '../data/real/'

In [11]:
data=loadmat('wine')
X,y=data['X'],data['y']

In [12]:
X.shape,y.shape

((129, 13), (129, 1))

In [14]:
X,y=csv_dataset('diabetes',os.getcwd())

/diabetes 

[numero elementi = 85916]
[percentage outliers = 0.096582708692211]
[number features = 4]
[number outliers = 8298]


## Deep Isolation Forest (DIF)